# A model for extracting density profiles across liquid interfaces from synchrotron radiation XPS data†


Our model only extracts information that is  consistent  with  a  given  data  set. It is as accurate and precise in the density profiles it extracts from the XPS data, as the data it inverts.

In case you want to extract density profiles from your own data, change variables below in the first code cell.
max_molar, bulk_density, list_of_energies, list_of_intensities, x1_list.

After that  run the cell via button "Run" above or by Shift-Enter

In [13]:
#the first code cell

#plotting library
from matplotlib import cm
import matplotlib.patches as patches
import matplotlib.pyplot as plt
#integration library
from scipy.integrate import simps
from scipy.interpolate import interp1d
#working with arrays
import numpy as np


#Data from I. Jordan, A. Beloqui  Redondo, M.A. Brown,  D. Fodor,M. Staniuk, 
#A. Kleibert, H. J. Wörner, J. B. Giorgi and J. A.Van Bokhoven, 
#Chemical Communications,  2014,50,  4242–4244.

max_molar = 46.11 #indicate maximum molar density
bulk_density = 3.22 #indicate bulk density
#List of energies at which intesities were measured, should be integers
list_of_energies = np.array((110, 210, 310, 435, 580, 810))    
#List of intensities, recieved at each corresponding energy           
list_of_intensities = np.array((0.0058, 0.0174, 0.0478, 0.145, 0.304, 1))
# list of x1 values, avoiding regions. Must be specefied by hands. 
#For example a good start for x1_list would be [0, 1, 2, 3, 4]
x1_list = [3, 4, 5]


Now run the cell below. If you feel curious or want to change accuracy our other points otherwise you can leave it as it is.

In [ ]:
# the second code cell

#needed functions

def experimental_fit(x):
    #fitted to experimental values 10.1103/PhysRevLett.111.173005
    # for cylindrical liquid jet we should include 2/pi 
    return 0.67151899*np.exp(0.00197393*x)


def exponential_decay(photon_energy, decay_range, function):
    return np.exp(-decay_range/function(photon_energy))


def get_all_possible_combinations(a):
    '''
    Input: an array. Returns an array of ratios of all elements
    '''
    z = np.array([])
    for i,j in enumerate(a):       
        z = np.append(z, j/a[i+1:])
    return z


def fitness_function(x, y, xs, intensities_ratios, decay_functions):
    '''
    calculates errors for provided energies, 
    returns one max error for all energies
    '''
    #get coordinates of a candidate
    spl_surf = interp1d(x, y, kind='previous')
    density = spl_surf(xs)
    modeled_intesities = np.array([simps(density*decay_function, xs) \
    for decay_function in decay_functions])
    modeled_intesities_ratios = get_all_possible_combinations(modeled_intesities)
    return np.amax(100*np.abs(1-modeled_intesities_ratios/intensities_ratios))


x0 = 0 #beginning of the liquid, x coordinate of point A
x3 = 50 #depth_to_invistigate, x coordinate of point F
range_splitting = 10001 #for better precision can be increased
xs = np.linspace(x0, x3, range_splitting)#
    

intensities_ratios = get_all_possible_combinations(list_of_intensities)
decay_functions = [exponential_decay(energy, xs, experimental_fit) for energy in list_of_energies]

#list of value for y0 coordinate (y coordinate for point A and B).
#Presence of species in the avoiding region. For example [0, 0.1, 1]
y0_list = [0]     

for step, y0 in enumerate(y0_list):
    print('Figure {} out of {}'.format(step+1, len(y0_list)))  
    fig, axs = plt.subplots(2, len(x1_list), gridspec_kw={'wspace': 0.05, 'hspace': 0.35},sharey='row',sharex='row')
    # starts calculation for one avoiding region
    for i, x1 in enumerate(x1_list):
        # initially create a grid for possible candidates (SiPs)
        x2_range=5.1+x1_list[-1]
        #grid in x axis
        grid_size_x = 0.1 #0.7 for faster results
        X = np.arange(x1, x2_range, grid_size_x)
        #range of y1 coordinates
        Y = np.linspace(y0,max_molar,50)
        #keep errors inside a list Z_temp
        errors_temp = []
        #evaluate each SiP 
        for y1 in Y:
            #y coordinates of a SiP
            y = [y0, y0, y1, y1, bulk_density, bulk_density] 
            for x2 in X:
                #x coordinates of a SiP
                x = [x0, x1, x1, x2, x2, x3]
                #provide coordinates of SiP to evaluate fit to intesities
                error = fitness_function(x, y, xs, intensities_ratios, decay_functions) 
                #write errors for SiP                   
                errors_temp.append(error)
        #make a proper shape for visualization
        errors = np.reshape(errors_temp, (Y.shape[0],X.shape[0]))
        #find best result and its coordinates
        best_result = np.where(errors== np.amin(errors))
        x2_best = X[best_result[1][0]]
        y1_best = Y[best_result[0][-1]]
        #creat best SiP
        best_ind = [[x0, x1, x1, x2_best, x2_best, x3], 
                    [y0, y0, y1_best, y1_best, bulk_density, bulk_density]]    
        spl_best = interp1d(best_ind[0], best_ind[1],kind='previous')
        
        '''
        Below Simulating intensities of best candidate through energy range
        in order to get "Intesity profile"
        '''
        
        energy_range = [i for i in range(100,910,1)] 
        simulated_intesities_of_best_SiP = []
        simulated_intensity_for_experim_energy = []
        
        for energy in energy_range:
            # for each energy make proper exponential decay
            exp_function_to_partition = exponential_decay(energy, xs, experimental_fit)
            #integrated best SiP*decay 
            simps_01 = simps(spl_best(xs)*exp_function_to_partition, xs)
            simulated_intesities_of_best_SiP.append(simps_01)
            #it is needed for normalization
            if energy in list_of_energies:
                simulated_intensity_for_experim_energy.append(simps_01)
        
        #first koef is to align intensities
        norm_koef = np.mean(list_of_intensities/
                            np.array(simulated_intensity_for_experim_energy))
        #second koef is to normalize to unity
        norm_koef_2 = np.amax(list_of_intensities/norm_koef)           
        int_normolized = list_of_intensities/norm_koef/norm_koef_2
        
        '''
        Below codes relates only to plotting, no more calculations.
        '''
        #Some code should be specified before loop. But we wanted to devide 
        #calculations and plotting to ease understanding.
        #creates colormap
        cmap=cm.get_cmap('hot', 7)
        cmap_colors=[cm.get_cmap('hot', 7)(i) for i in range(0,cmap.N)]
        
        ax0 =axs[0][i]
        colorbar = ax0.contourf(X, Y, errors, 50 ,levels=[0,5,10,20,30,50,100], 
                           colors=cmap_colors)
        # Create a Rectangle patch for avoiding region
        rect = patches.Rectangle((0,y0), width=x1, height=Y[-1]-y0, fill=True, 
                                 color='blue', alpha=0.2, 
                                 label='Avoiding region',linewidth=0)
        ax0.add_patch(rect)
        # to avoid best candidates with zero width of the peak
        if x1 == 0:
            if best_ind[0][0]==best_ind[0][1] and best_ind[0][0]==best_ind[0][2] \
            and best_ind[0][0]==best_ind[0][3]and best_ind[0][0]==best_ind[0][4]:
                ind_to_plot_x = best_ind[0][4:]
                ind_to_plot_y = best_ind[1][4:]
            else:
                ind_to_plot_x = best_ind[0][2:]
                ind_to_plot_y = best_ind[1][2:]

        elif best_ind[0][2]==best_ind[0][3]:
            ind_to_plot_x = [best_ind[0][i] for i in [0,1,4,5]]
            ind_to_plot_y = [best_ind[1][i] for i in [0,1,4,5]] 
    
        else:
            ind_to_plot_x = best_ind[0]
            ind_to_plot_y = best_ind[1]
        
        ax0.plot(ind_to_plot_x, ind_to_plot_y, color='lime', lw=3, 
                 linestyle='dashed', label='Best candidate')
        print('Best candidate', 'x = ', best_ind[0], 'y = ', best_ind[1])
        # writes text avoiding region in nm
        if x1!=0:
            ax0.text(x1/2, y1/2, 'x1 = {:.1f}nm'.format(x1), 
                     horizontalalignment='center', 
                     verticalalignment='center', rotation=90)
            
        ax1 = axs[1][i]
        ax1.plot(energy_range, simulated_intesities_of_best_SiP/norm_koef_2, 
                 color='limegreen', linestyle='dashed', 
                 label='Intensity profile\nof best candidate')
        ax1.scatter(list_of_energies, int_normolized, 
                    label='Intensities from\nexperiment')
        ax1.text(1, 0, 
                 r'$\mathregular{RE_{max}=}$'+'{:.1f}%'.format(np.amin(errors)), 
                 horizontalalignment='right', verticalalignment='bottom', 
                 transform = ax1.transAxes)        
        ax1.set_xticks([100*i for i in range(1,10,2)])
        print('SubFigure {} out of {}'.format(i+1, len(x1_list)))
        
    # add a big axis, hide frame, to make x labels in the middle
    fig.add_subplot(111, frameon=False)
    # hide tick and tick label of the big axis
    plt.tick_params(labelcolor='none', top=False, bottom=False, 
                    left=False, right=False)
    plt.xlabel("Kinetic energy (eV)")    
    fig.add_subplot(211, frameon=False)
    plt.tick_params(labelcolor='none', top=False, bottom=False, 
                    left=False, right=False)
    plt.xlabel("Depth (x2, nm)")   
    plt.ylabel('Molar concentration\n(y1, mol/L)')
    
    axs[0][0].set_yticks([y0]+[10,20,30,40,50])
    axs[0][0].set_xticks([i for i in range(0,20,2)])
    axs[0][0].set_yticklabels(['y0={:.2f}'.format(y0)]+['10','20','30','40','50'])
    axs[0][0].axis((-0.01, X[-1], -0.01, Y[-1]+0.4))
    
    axs[1][0].set_yticks([])
    axs[1][0].set_ylabel('XPS signal (a.u.)')
    axs[1][0].axis((0, 1000, -0.2, 2))
    
    axs[0][-1].legend(bbox_to_anchor=(1.05, 1), loc='upper left', 
                      borderaxespad=0.)
    axs[1][-1].legend(bbox_to_anchor=(1.05, 0.4), loc='upper left', 
                      borderaxespad=0.)

    inc=3.64+len(x1_list)*2.34
    fig.subplots_adjust(left=0.08*13/inc, right=(1-0.2*(13/inc)), 
                        top=0.96, bottom=0.08)
    cbar_ax = fig.add_axes([(1-0.17*(13/inc)), 0.29, 0.03*13/inc, 0.5])
    plt.colorbar(colorbar, cax=cbar_ax, orientation='vertical', label='Error %')
    fig.set_size_inches(inc, 7)
    plt.subplots_adjust(wspace=0.05, hspace=0.35)
    plt.show()                                                                              


Figure 1 out of 1
Best candidate x =  [0, 3, 3, 4.300000000000001, 4.300000000000001, 50] y =  [0, 0, 0.0, 0.0, 3.22, 3.22]
SubFigure 1 out of 3
